# Llama 2 70B 에이전트/도구 사용 예제 

이 Jupyter 노트북은 EasyLLM에서 Llama 2 70B 모델과 함께 에이전트용 도구를 사용하는 방법에 대한 예제를 제공합니다. 여기에는 출력 구문 분석, 도구 실행 및 결과 구문 분석을 포함하여 LLM과 함께 도구를 사용하는 방법에 대한 예제가 포함되어 있습니다. 매우 단순화된 예제입니다. 에이전트에 관심이 있다면 [langchain](https://python.langchain.com/docs/get_started/introduction.html) 또는 [ReAct 패턴](https://www.promptingguide.ai/techniques/react)을 확인해야 합니다.


## LLM이 도구를 사용해야 하는 이유는 무엇입니까?
LLM의 가장 일반적인 과제 중 하나는 학습 데이터의 최신성 및 특수성 부족을 극복하는 것입니다. 답변이 오래되었을 수 있으며 지식 기반의 방대한 다양성으로 인해 환각을 일으키기 쉽습니다. 도구는 LLM이 기존 지식 기반 및 내부 API를 활용하는 제어된 컨텍스트 내에서 답변할 수 있도록 하는 훌륭한 방법입니다. LLM을 의도한 답변으로 프롬프트 엔지니어링하려고 하는 대신 정보를 동적으로 호출하고 구문 분석하여 고객에게 제공하는 도구에 대한 액세스를 허용합니다.

LLM에 도구에 대한 액세스 권한을 제공하면 검색 엔진, API 또는 자체 데이터베이스에서 직접 컨텍스트를 사용하여 질문에 답변할 수 있습니다. 도구에 액세스할 수 있는 LLM은 직접 답변하는 대신 관련 정보를 수집하기 위한 중간 단계를 수행할 수 있습니다. 도구를 조합하여 사용할 수도 있습니다. 예를 들어 언어 모델은 검색 도구를 사용하여 정량적 정보를 조회하고 계산기를 사용하여 계산을 실행하도록 만들 수 있습니다.

In [ ]:
# 필요한 경우 EasyLLM Python 라이브러리의 최신 버전으로 설치 및/또는 업그레이드합니다.
%pip install --upgrade easyllm 

오픈 LLM을 에이전트처럼 작동하게 하거나 도구를 사용하게 하는 것은 매우 어렵습니다. 하지만 Llama 2 70B를 사용하면 이제 가능합니다. 어떻게 실행할 수 있는지 봅시다!

## Llama 2 70B와 함께 도구를 사용하는 기본 예제

기본적으로 우리는 추상적인 도구인 `계산기` 하나만 사용할 것입니다. 우리 모델은 계산기를 사용하여 수학적 연산을 실행할 수 있습니다. 쉽게 하기 위해 모델이 무엇을 해야 하는지 더 잘 이해할 수 있도록 몇 가지 소수 샷 예제를 제공합니다.
_참고: 이것은 [pinecone의 예제](https://github.com/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-70b-chat-agent.ipynb)에서 수정되었습니다._

In [41]:
system_message = """조수는 광범위한 작업을 지원하도록 설계된 전문가 JSON 빌더입니다.

조수는 사용자에게 응답하고 "action" 및 "action_input" 매개변수가 포함된 JSON 문자열을 사용하여 도구를 사용할 수 있습니다.

조수의 모든 통신은 이 JSON 형식을 사용하여 수행됩니다.

조수는 동일한 "action" 및 "action_input" JSON 형식으로 도구 사용 지침을 사용자에게 응답하여 도구를 사용할 수도 있습니다. 조수가 사용할 수 있는 도구는 다음과 같습니다.

- "계산기": 수학 관련 질문에 답해야 할 때 유용합니다.
  - 계산기 도구를 사용하려면 조수는 다음과 같이 작성해야 합니다.
    ```json
    {{"action": "계산기",
      "action_input": "4+4"}}
    ```

다음은 조수와 사용자 간의 이전 대화 중 일부입니다.

사용자: 안녕하세요, 오늘 어떠세요?
조수: ```json
{{"action": "최종 답변",
 "action_input": "저는 괜찮아요, 당신은 어떠세요?"}}
```
사용자: 저는 아주 좋아요, 4의 제곱근은 무엇인가요?
조수: ```json
{{"action": "계산기",
 "action_input": "sqrt(4)"}}
```
결과: 2.0
조수: ```json
{{"action": "최종 답변",
 "action_input": "답은 2인 것 같아요!"}}
```
사용자: 감사합니다. 4의 2제곱이 무엇인지 알려주시겠어요?
조수: ```json
{{"action": "계산기",
 "action_input": "4**2"}}
```
결과: 16.0
조수: ```json
{{"action": "최종 답변",
 "action_input": "답은 16인 것 같아요!"}}
```

다음은 조수와 사용자 간의 최근 대화입니다."""

도구에 대한 정보를 담고 있는 시스템 메시지 외에도 사용자의 입력을 포함하고 모델에게 도구를 사용할지 여부를 알려주는 사용자 템플릿을 만들어야 합니다.

In [42]:
prompt = f"{system_message}\n\n기존 도구를 사용하고 'action' 및 'action_input' 값이 있는 JSON 개체로 응답합니다. \n사용자: {{user_input}}"

이제 둘 다 결합하여 `easyllm`을 사용하여 요청을 만들어 보겠습니다.

In [43]:

from easyllm.clients import huggingface

# 환경 변수를 사용하지 않고 구성 변경
huggingface.prompt_builder = "llama2"
# huggingface.api_key="hf_xxx"

def agent(prompt):
  response = huggingface.Completion.create(
      model="meta-llama/Llama-2-70b-chat-hf",
      prompt=prompt,
      temperature=0.1,
      max_tokens=128,
      stop=["```\n","결과: "],
      debug=False,
  )  
  return response["choices"][0]["text"]


이제 질문을 시작할 수 있습니다.

In [44]:
output = agent(prompt.format(user_input="안녕하세요, 오늘 어떠세요?"))
output

' 조수: ```json\n{"action": "최종 답변",\n "action_input": "저는 괜찮아요, 당신은 어떠세요?"}\n```'

수학 문제를 물어보면 어떻게 될까요? 

In [45]:
output = agent(prompt.format(user_input="4 곱하기 2는 무엇인가요?"))
output

' 조수: ```json\n{"action": "계산기",\n "action_input": "4*2"}\n```\n'

좋아요! 작동합니다! 도구를 올바르게 선택합니다. 이제 작동하도록 하려면 출력을 구문 분석하고 계산기의 경우 실행해야 합니다.

In [46]:
import json
import re

def parser(input):
    pattern = r'```json\n(.*?)```'
    match = re.search(pattern, input, re.DOTALL)
    if not match:
        raise ValueError("출력을 구문 분석할 수 없습니다.")
    
    parsed_data = json.loads(match.group(1))
    return parsed_data

    

In [47]:
output = parser(output)
output

{'action': '계산기', 'action_input': '4*2'}

좋아요, 이제 파이썬의 `eval` 함수를 사용하여 실행해 봅시다.

In [48]:
def use_tool(tool,tool_input):
  if tool == "계산기":
    return eval(tool_input)
  else:
    raise Exception("알 수 없는 도구: " + tool)

좋아요, 이제 모든 것을 결합하고 계산기 결과를 에이전트에 다시 추가해 봅시다.

In [73]:
def use_calculator(input, first_call=True):
  if first_call:
    input_prompt = prompt.format(user_input=input)
  else:
    input_prompt = input
  # 에이전트 호출하기
  response = agent(input_prompt)
  # 가능한 경우 출력 구문 분석 
  parsed = parser(response)
  # 출력이 최종 답변인지 아니면 도구인지 확인합니다.
  if parsed["action"] == "최종 답변":
    return parsed["action_input"]
  # 그렇지 않으면 도구를 사용해 봅니다.
  tool_output = use_tool(parsed["action"], parsed["action_input"])
  
  # 에이전트에 메시지 추가
  next_prompt = f"{input_prompt}\n{response}\n응답: {tool_output}"
  # 도구의 출력으로 에이전트를 재귀적으로 호출합니다.
  return use_calculator(next_prompt, False)
  
  

In [75]:
use_calculator("19 * 11은 무엇인가요?")

'답은 209인 것 같아요!'